## About
This is a notebook for calculating the following statistics in a map:
- producer's accuracies and user's accuracies by class,
- overall accuracy, and
- area estimates. 
Each statistic is accompanied by a 95% confidence interval. 
To do this, we follow the notation and calculations in Olofsson et al. 

The data input needs to be:
1. a csv of the points assessed with two columns: map_class and ref_class. Map class is the classification of the point in the map, ref_class is the "ground truth" classification of the point. 

2. a csv with the number of pixels per class in the map.

Alternatiely, the notebook can be run using the confusion matrix (table 8) in the Olofsson et al. paper. 

## Reference

Pontus Olofsson, Giles M. Foody, Martin Herold, Stephen V. Stehman, Curtis E. Woodcock, Michael A. Wulder, Good practices for estimating area and assessing accuracy of land change, Remote Sensing of Environment,Volume 148, 2014, Pages 42-57, ISSN 0034-4257, https://doi.org/10.1016/j.rse.2014.02.015.  (https://www.sciencedirect.com/science/article/pii/S0034425714000704)

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

### Load Data

To run the example in Olofsson et al., do not import any data and do not execute the cells in this section. 
Instead, go to the next section and execute the cell having the Olofsson et al. data directly.

In [2]:
# load csv with validation points
df = pd.read_csv(os.path.join(os.getcwd(),
                              'validation_data_salt13_p30_map',
                              'salt13_p30_map_and_reference_classes.csv'))
df.head()

In [3]:
#load counts of pixels per class in map
pix_counts = pd.read_csv(os.path.join(os.getcwd(),
                                                'validation_data_salt13_p30_map',
                                                'salt13_p30_2020_combined_pixel_counts_total.csv'))
pix_counts = pix_counts.to_numpy()[0]
pix_counts

In [4]:
# counts by reference class
print('Points in each reference class')
print(np.unique(df.ref_class, return_counts=True), '\n')

# counts by map class: these should match the counts given by the stratified sample design
print('Points in each map class')
print(np.unique(df.map_class, return_counts=True))

In [16]:
# specify number of classes in the map
n_classes = 4

### Confusion Matrix

Here we create a confusion matrix $n$ such that 

$n_{i,j}$ = number of points predicted as $i$, known to be $j$, 

which is equivalent to

$n_{i,j}$ = number of points that have map class as $i$ and reference class $j$.

In [6]:
# CALCULATE CONFUSION MATRIX FOR IMPORTED DATA

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# using confusion_matrix directly we get a matrix C such that
# C_{i,j} = known to be i, predicted as  j 
# The notation in the paper is 
# n_{i,j} = predicted as i, known to be j 
# so we need to take the transpose

n = confusion_matrix(df.ref_class, df.map_class, labels=range(n_classes)).T
n

In [7]:
# USE EXAMPLE TO RUN NOTEBOOK

# Olofsson et al. data - Table 8

pix_counts = np.array([200000, 150000, 3200000, 6450000])
print(pix_counts)

n = np.array([
    [66, 0,  5,   4],
    [0,  55, 8,   12],
    [1,  0,  153, 11],
    [2,  1,  9,   313]])
n

[ 200000  150000 3200000 6450000]


array([[ 66,   0,   5,   4],
       [  0,  55,   8,  12],
       [  1,   0, 153,  11],
       [  2,   1,   9, 313]])

### Notation
Throughout the following, let $p_{ij}$ be the (true) fraction of the map that has map class $i$ and reference class $j$. 

### User's Accuracy
The user's accuracy of class $i$ is the fraction of the area mapped as class $i$ that has reference class $i$, this is (Olofsson et al. eq 2):
$$U_i = \frac{p_{ii}}{p_{i\cdot}}.$$
This is equivalent to the precision of class $i$. For example, when there are two classes (positive and negative) the user's accuracy of the positive class is the same as the precision of the true class (TP/(TP + FP)).

To estimate the $U_i$'s from the points sample we have that
$$\hat{U}_i = \frac{\hat{p}_{ii}}{\hat{p}_{i\cdot}},$$
where $\hat{p}_{ij}$ are the estimations of $p_{ij}$ from the sample. 
For stratified random sampling in which the sampling strata correspond to the map classes we have that
$$\hat{p}_{ij} = W_i \frac{n_{ij}}{n_{i\cdot}},$$
where
- $W_i$ is the fraction of the map's area classified  as class i,
- $n_{ij}$ is number of points with map class $i$, known to be reference class $j$ (entires in the confusion matrix), and
- $n_{i\cdot}$ is the number of points with map class $i$ (row sums in confusion matrix).

Notice that the user's accuracy can be simplifeid to
$$\hat{U}_i = \frac{n_{ii}}{n_{i\cdot}},$$
this is the formula implemented in the code.

For user's accuracy of map class $i$, the estiamted variance is (Olfosson et al. eq. 6):
$$\hat{V}(\hat{U}_i) = \frac{\hat{U}_i (1-\hat{U}_i)}{n_{i\cdot}-1}.$$

**NOTE:** We calculate the user's accuracies first since these are needed to calculate the approximate variance of the overall accuracy. 

### Variance, Standard Error & Confidence Intervals
Recall that the square root of the estimated variance results in the standard error of the estimator. For example, in the case of the estimated overall accuracy of the map $\hat{O}$ we have that $\hat{S}(\hat{O}) = \sqrt{\hat{V}(\hat{O})}$ (see Olofsson et al. eq. 5). 

Also, the standard error is used to get confidence intervals for the estimated statistic:the 95% confidence interval is estimated as $\hat{O} \pm 1.96 \hat{S}(\hat{O}) = \hat{O} \pm 1.96\  \sqrt{\hat{V}(\hat{O})}$.

In [8]:
# -------------------------------------
# points in sample that had class i in map (predicted as i, any true class j)
# these will also be used in overal accuracy and producer's accuracies
n_idot = [sum(n[i,:]) for i in range(n_classes)]

# -------------------------------------
# estimated users' accuracy (precision for each class: TP/(TP+FP))
U_hat = [n[i,i] / n_idot[i] for i in range(n_classes)]

var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)]

# -------------------------------------
print("user's accuracies:", [x*100 for x in U_hat])
print("user's accuracies confidence interval:", 1.95*np.sqrt(var_U_hat)*100)

user's accuracies: [88.0, 73.33333333333333, 92.72727272727272, 96.3076923076923]
user's accuracies confidence interval: [ 7.3663222  10.02429481  3.95425872  2.04287379]


### Overal Accuracy

Let $O$ be the (true) accuracy of the map, and $\hat{O}$ its estimation from the sample. Then, following Olofsson et al., section 4.3, we have that 
$$\hat{O} = \sum_{i=1}^q \hat{p}_{ii},$$
where $q$ is the number of classes in the map and $\hat{p}_{ii}$ is the estimation of $p_{ij}$, the (true) fraction of the area in the map that was classified as class $i$ and has reference class $j$. 

For overall accuracy, the estimated variance is (Olofsson et al. eq 5):
$$\hat{V}(\hat{O}) = \sum_{i=1}^q \frac{W_i^2 \hat{U}_i (1-\hat{U}_i)}{n_{i\cdot}-1}.$$

In [9]:
# total number of pixels in the map
total_pix = sum(pix_counts)

# list with the fractions of area in map mapped as each class
W = [pix_counts[i]/ total_pix for i in range(n_classes)]      

# -------------------------------------
# overall accuracy
O_hat = sum([W[i]*n[i,i]/n_idot[i] for i in range(n_classes)])
print('overall accuracy:', O_hat*100)

# -------------------------------------
var_O_hat = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)])

# std error of estimated overall accuracy
print('overall accuracy confidence interval:', 1.95*np.sqrt(var_O_hat)*100, '\n')

overall accuracy: 94.65118881118882
overall accuracy confidence interval: 1.8389313570398378 



### Producer's Accuracy
The producer's accuracy of class $i$ is the fraction of the (true) area with reference class $i$ that is actually mapped as class $j$, this is (Olofsson et al. eq 3):
$$P_j = \frac{p_{jj}}{p_{\cdot j}}.$$
This is equivalent to the sensitiviy of class $j$. For example, when there are two classes (positive and negative) the producer's accuracy of the positive class is the same as the sensitivy of the true class (TP/(TP + FN)).

To estimate the $P_i$'s from the points sample we have that
$$\hat{P}_j = \frac{\hat{p}_{jj}}{\hat{p}_{\cdot j}},$$
where the $\hat{p}_{ij}$ are as before.

For the producer's accuracy of class $j$ the estimated variance is given by (Olofsson et al. eq 7):

$$\hat{V}(\hat{P}_j) = 
\frac{1}{\hat{N}_{\cdot j}^2} 
\left( 
\frac{N_{j \cdot}^2 (1 - \hat{P}_j)^2 \hat{U}_j (1-\hat{U}_j)}{n_{j \cdot} -1}  
+
\hat{P}_j^2
\sum_{i\neq j}^q 
\frac{N_{i\cdot}^2}{n_{i \cdot} - 1} 
\frac{n_{ij}}{n_{i \cdot}} 
\left( 1 - \frac{n_{ij}}{n_{i \cdot}} \right)
\right),$$
where
- $N_{j \cdot}$ is the number of pixels in the map with map class $j$,
- $n_{j\cdot}$ is the number of sample points with map class $j$, and
- $\hat{N}_{\cdot j} = \sum_{i=1}^q N_{i\cdot}\frac{n_{ij}}{n_{i\cdot}}$ is the estimated number of pixels with reference class $j$.



In [10]:
p_hat_dotj = []
# estimated producer's accuracy (sensitiviy for each class TP/(TP+FN))
P_hat = []  

for j in range(n_classes):
    # list of p_hat_ij with fixed j
    p_hat_ij = [ W[i]*n[i,j]/n_idot[i] for i in range(n_classes) ]
    p_hat_dotj.append(sum(p_hat_ij))  # equation (9)


P_hat= [ (W[j]*n[j,j]/n_idot[j]) / p_hat_dotj[j] for j in range(n_classes)]
# -------------------------------------
print("producer's accuracies:", [x*100 for x in P_hat])

# -------------------------------------
# -------------------------------------
# VARIANCE
# notice N_jdot is pixel_counts[j]
N_hat_cdotj = []
for j in range(n_classes):
    summands = [ pix_counts[i] * n[i,j]/n_idot[i] for i in range(n_classes)]
    N_hat_cdotj.append(sum(summands))

# -------------------------------------
summand1 = [ (pix_counts[j]**2) * ((1-P_hat[j])**2) * U_hat[j] * (1-U_hat[j]) / (n_idot[j] - 1) 
            for j in range(n_classes)]

# -------------------------------------
summand2 = []
for j in range(n_classes):
    inner = []
    for i in range(n_classes):
        if i!=j:
            inner.append( (pix_counts[i]**2) /(n_idot[i]-1) * (n[i,j])/(n_idot[i]) * ( 1 - n[i,j]/n_idot[i]) ) 
    summand2.append((P_hat[j]**2) * sum(inner))

# -------------------------------------
var_P_hat = [1/(N_hat_cdotj[j]**2) *  (summand1[j] + summand2[j]) for j in range(n_classes)]

# -------------------------------------
# -------------------------------------
print("producer's accuracies confidence interval:", 1.95*np.sqrt(var_P_hat)*100, '\n')

producer's accuracies: [74.86614048308412, 84.71563981042654, 93.45089085796926, 96.16089928314558]
producer's accuracies confidence interval: [21.22215374 25.31103589  3.41492981  1.82678542] 



### Area Estimates

For stratified random sampling when the map classes are the strata, we have that an estimator of the proportion of area of class $j$ is (Olofsson et al. eq. 9):
$$ \hat{p}_{\cdot j} = \sum_{i=1}^q W_i \frac{n_{ij}}{n_{i\cdot}}.$$

For this estimator of area proportion per class, the standard error is estimated by (Olofsson et al. eq 10):
$$S(\hat{p}_{\cdot j}) =  
\sqrt{
\sum_{i=1}^q W_i^2 \frac{ \frac{n_{ij}}{n_{i\cdot}} \left(1 -  \frac{n_{ij}}{n_{i\cdot}} \right)}{n_{i \cdot}-1}
}.$$

The estimated area of class $j$ is
$$\hat{A}_j = A \times \hat{p}_{\cdot k},$$
where $A$ is the total are of the map. 
The standard error for the area is given by (Olofsson et al. eq 11):
$$ S(\hat{A}_j) = A \times S(\hat{p}_{\cdot j}).$$

In [11]:
# PERCENTAGE OF AREA ESTIMATION
# we had calculated the area estimators before, they are used in producer's accuracy
print("percentage of area per class: \n", [x*100 for x in p_hat_dotj])

# -------------------------------------
# STD ERROR
SE_p_hat_dotj = []
for j in range(n_classes):
    summands = [ (W[i]**2) * (n[i,j]/n_idot[i]) * (1 -  (n[i,j]/n_idot[i]))/ (n_idot[i]-1) 
                for i in range(n_classes)]
    SE_p_hat_dotj.append(np.sqrt(sum(summands)))
    
print("confidence interval for percentage area per class:\n", [x*1.96*100 for x in SE_p_hat_dotj])


percentage of area per class: 
 [2.350862470862471, 1.2984615384615383, 31.752214452214456, 64.59846153846155]
confidence interval for percentage area per class:
 [0.6841815984519081, 0.4173140028226505, 1.723315144243176, 1.8090729280271949]


In [12]:

map_area = total_pix * (21158/235086)

approx_area_per_class = [map_area * p_hat_dotj[i] for i in range(n_classes)]
print("approx area per class (ha): \n", approx_area_per_class)

SE_area_per_class = [map_area * SE_p_hat_dotj[i] for i in range(n_classes)]
print("confidence interval for area per class (m^2):\n", [x*1.96 for x in SE_area_per_class])

approx area per class (ha): 
 [21158.02223803551, 11686.297453174257, 285773.4417957486, 581393.2982954193]
confidence interval for area per class (m^2):
 [6157.710055063028, 3755.8721794243975, 15510.026893093216, 16281.856431773647]


In [13]:
# # AREA ESTIMATION
# # in m^2, assuming a resolution of 0.5m per pixel side
# # (each pixel has an area of 0.25 m^2)
# map_area = total_pix * 0.25 

# approx_area_per_class = [map_area * p_hat_dotj[i] for i in range(n_classes)]
# print("approx area per class (m^2): \n", approx_area_per_class)

# SE_area_per_class = [map_area * SE_p_hat_dotj[i] for i in range(n_classes)]
# print("confidence interval for area per class (m^2):\n", [x*1.96 for x in SE_area_per_class])

In [14]:
# # in km^2, assuming a resolution of 0.5m per pixel side
# map_area = total_pix * 0.25 / (1000**2)

# approx_area_per_class = [map_area * p_hat_dotj[i] for i in range(n_classes)]
# print("approx area per class (km^2): \n", approx_area_per_class)

# SE_area_per_class = [map_area * SE_p_hat_dotj[i] for i in range(n_classes)]
# print("confidence interval for area per class (km^2):\n", [x*1.96 for x in SE_area_per_class])